# (Project Proposal Title)

## Introduction

(Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal)

(Clearly state the question you will try to answer with your project)

(Identify and describe the dataset that will be used to answer the question)

## Preliminary Exploratory Data Analysis

In [1]:
# read data

In [2]:
# clean, wrangle data

In [3]:
# Summarize training data

In [4]:
# Visualize training data

## Methods
(Explain how you will conduct either your data analysis and which variables/columns you will use)
- Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?

(Describe at least one way that you will visualize the results)

## Expected outcomes and significance:
### What do you expect to find?
TBD
### What impact could such findings have?
TBD
### What future questions could this lead to?
TBD